In [1]:
import sys
sys.path.append("../")

import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim

import torchdiffeq

from tensorboard_utils import Tensorboard
from tensorboard_utils import tensorboard_event_accumulator

import transformer.Constants as Constants
from transformer.Layers import EncoderLayer, DecoderLayer
from transformer.Modules import ScaledDotProductAttention
from transformer.Models import Decoder, get_attn_key_pad_mask, get_non_pad_mask, get_sinusoid_encoding_table
from transformer.SubLayers import PositionwiseFeedForward

import dataset

import model_process
import checkpoints
from node_transformer import NodeTransformer

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook  
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("Torch Version", torch.__version__)

%load_ext autoreload
%autoreload 2

Torch Version 1.1.0


In [2]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


In [3]:
data = torch.load("/home/mandubian/datasets/multi30k/multi30k.atok.low.pt")

In [4]:
max_token_seq_len = data['settings'].max_token_seq_len
print(max_token_seq_len)

52


In [5]:
train_loader, val_loader = dataset.prepare_dataloaders(data, batch_size=128)

### Create an experiment with a name and a unique ID

In [6]:
exp_name = "transformer_6_layers"
unique_id = "2019-06-20_2330"


### Create Model

In [7]:
model = None

In [8]:
from odeint_ext_adams import *

src_vocab_sz = train_loader.dataset.src_vocab_size
print("src_vocab_sz", src_vocab_sz)
tgt_vocab_sz = train_loader.dataset.tgt_vocab_size
print("tgt_vocab_sz", tgt_vocab_sz)

if model:
    del model

RTOL = 0.01
LR=1e-4
N_HEAD=8
N_LAYERS=6
METHOD='dopri5-ext'
HAS_NODE_ENCODER=False
HAS_NODE_DECODER=False
HAS_SEPARATED_NODE_DECODER=False
ADD_TIME=False

checkpoint_desc = {
    #"rtol":RTOL,
    "lr":LR,
    "n_layers":N_LAYERS,
    "n_head":N_HEAD,
    #"method":METHOD,
    "has_node_encoder":HAS_NODE_ENCODER,
    "has_node_decoder":HAS_NODE_DECODER,
    "has_separated_node_encoder":HAS_SEPARATED_NODE_DECODER,
    "add_time": ADD_TIME,
}

model = NodeTransformer(
    n_src_vocab=max(src_vocab_sz, tgt_vocab_sz),
    n_tgt_vocab=max(src_vocab_sz, tgt_vocab_sz),
    len_max_seq=max_token_seq_len,
    #emb_src_tgt_weight_sharing=False,
    #d_word_vec=256, d_model=256, d_inner=1024,
    n_layers=N_LAYERS,
    n_head=N_HEAD, method=METHOD, rtol=RTOL, atol=RTOL,
    has_node_encoder=HAS_NODE_ENCODER, has_node_decoder=HAS_NODE_DECODER,
    has_separated_node_decoder=HAS_SEPARATED_NODE_DECODER, add_time=ADD_TIME,
)

model = model.to(device)

src_vocab_sz 9795
tgt_vocab_sz 17989


### Create Tensorboard metrics logger

In [9]:
tb = Tensorboard(exp_name, unique_name=unique_id, output_dir="../runs")

Writing TensorBoard events locally to ../runs/transformer_6_layers_2019-06-20_2330


### Create basic optimizer

In [10]:
optimizer = optim.Adam(model.parameters(), lr=LR, betas=(0.9, 0.995), eps=1e-9)

#optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


### Train

In [11]:
# Continuous space discretization
timesteps = np.linspace(0., 1., num=2)
timesteps = torch.from_numpy(timesteps).float()

EPOCHS = 100
LOG_INTERVAL = 5

#from torch import autograd
#with autograd.detect_anomaly():
model_process.train(
    exp_name, unique_id,
    model, 
    train_loader, val_loader, timesteps,
    optimizer, device,
    epochs=EPOCHS, tb=tb, log_interval=LOG_INTERVAL,
    start_epoch=0,
    checkpoint_desc= checkpoint_desc,
    #best_valid_accu=state["acc"],
)

Loaded model and timesteps to cuda


[ Epoch 0 ]
Adding group train to writers (dict_keys([]))
[Training]  loss: 7.2235593511588565, ppl:  1371.36154, accuracy: 16.599 %, elapse: 61566.379ms
Adding group eval to writers (dict_keys(['train']))
[Validation]  loss: 5.975024863551293,  ppl:  393.47788, accuracy: 27.932 %, elapse: 886.642ms
Checkpointing Validation Model...


[ Epoch 1 ]
[Training]  loss: 6.131624574610461, ppl:  460.18316, accuracy: 30.963 %, elapse: 61741.634ms
[Validation]  loss: 5.2839826120245,  ppl:  197.15350, accuracy: 33.047 %, elapse: 886.225ms
Checkpointing Validation Model...


[ Epoch 2 ]
[Training]  loss: 5.611737301748947, ppl:  273.61918, accuracy: 35.105 %, elapse: 61829.323ms
[Validation]  loss: 4.799503687417168,  ppl:  121.45013, accuracy: 36.256 %, elapse: 889.352ms
Checkpointing Validation Model...


[ Epoch 3 ]
[Training]  loss: 5.249176728696815, ppl:  190.40945, accuracy: 38.443 %, elapse: 61881.548ms
[Validation]  loss: 4.434336158571531,  ppl:  84.29615, accuracy: 39.802 %, elapse: 885.400ms
Checkpointing Validation Model...


...

[ Epoch 41 ]
[Training]  loss: 2.339608342409426, ppl:  10.37717, accuracy: 80.619 %, elapse: 61573.939ms
[Validation]  loss: 3.126169720252167,  ppl:  22.78653, accuracy: 55.298 %, elapse: 887.470ms
Checkpointing Validation Model...


[ Epoch 42 ]
[Training]  loss: 2.3057704804396466, ppl:  10.03190, accuracy: 81.219 %, elapse: 61534.669ms
[Validation]  loss: 3.154124844970091,  ppl:  23.43252, accuracy: 54.925 %, elapse: 900.562ms


[ Epoch 43 ]
[Training]  loss: 2.2770802726710504, ppl:  9.74818, accuracy: 81.706 %, elapse: 61848.229ms
[Validation]  loss: 3.152339972195358,  ppl:  23.39073, accuracy: 55.033 %, elapse: 885.443ms


[ Epoch 44 ]


KeyboardInterrupt: 

In [19]:
model.encoder.encoder.rtol = 0.001
model.encoder.encoder.atol = 0.001
#model.decoder.decoder.rtol = 0.001
#model.decoder.decoder.atol = 0.001

### Restore best checkpoint (to restart past training)

In [11]:
state = checkpoints.restore_best_checkpoint(
    exp_name, unique_id, "validation", model, optimizer)

print("accuracy", state["acc"])
print("loss", state["loss"])
model = model.to(device)

Extracting state from checkpoints/node_transformer_separated_dopri5_multi30k_encoder_only_add_time_2019-06-20_1230_validation_best.pth
Loading model state_dict from state found in checkpoints/node_transformer_separated_dopri5_multi30k_encoder_only_add_time_2019-06-20_1230_validation_best.pth
Loading optimizer state_dict from state found in checkpoints/node_transformer_separated_dopri5_multi30k_encoder_only_add_time_2019-06-20_1230_validation_best.pth
accuracy 0.5886524822695035
loss 2.558568739534082
